In [38]:
# Dependencies
import pandas as pd

In [39]:
#READ / FORMAT COVID DATA
cdc_covid_raw_data = "Resources/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
cdc_covid_raw_data = pd.read_csv(cdc_covid_raw_data, encoding="UTF-8")
cdc_covid_clean_data = cdc_covid_raw_data.drop(['conf_cases','prob_cases','pnew_case','conf_death','prob_death','pnew_death','created_at','consent_cases','consent_deaths'], axis=1) #drop irrelvant info
cdc_covid_clean_data['submission_date'] = pd.to_datetime(cdc_covid_clean_data['submission_date']) #format date
cdc_covid_clean_data.head(3)

,submission_date,state,tot_cases,new_case,tot_death,new_death
0,2021-04-01,CA,3570660,2234,58090,154
1,2021-05-31,CA,3685032,644,62011,5
2,2020-02-06,NE,0,0,0,0


In [40]:
#READ / FORMAT BTS DATA
bts_trip_raw_data = "Resources/Trips_by_Distance.csv"
bts_trip_raw_data = pd.read_csv(bts_trip_raw_data, encoding="UTF-8")
bts_trip_state_data = bts_trip_raw_data.loc[(bts_trip_raw_data['Level']=='State')]
bts_trip_clean_data = bts_trip_state_data.drop(['State FIPS','County FIPS','County Name','Row ID','Week','Month'],axis='columns')
bts_trip_clean_data['Date'] = pd.to_datetime(bts_trip_clean_data['Date'])
bts_trip_clean_data.head(3)

,Level,Date,State Postal Code,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
887,State,2019-01-01,NV,753054.0,2272063.0,9139676.0,2999886.0,2181216.0,1039500.0,1378024.0,1108436.0,222837.0,93301.0,73578.0,24471.0,18427.0
888,State,2019-01-01,FL,4524666.0,16709556.0,55258300.0,16036792.0,14469767.0,6658624.0,7977612.0,6604304.0,2113009.0,796740.0,412224.0,95341.0,93887.0
889,State,2019-01-01,DC,241030.0,459278.0,3114055.0,1314825.0,833796.0,367547.0,344116.0,183744.0,42096.0,10730.0,8109.0,3476.0,5616.0


In [41]:
# MERGE / CLEAN DATAFRAMES
cdc_bts_merged_df = pd.merge(cdc_covid_clean_data, bts_trip_clean_data,  how='inner', left_on=['submission_date','state'], right_on = ['Date','State Postal Code'])
cdc_bts_merged_df = cdc_bts_merged_df.drop(['submission_date','State Postal Code'],axis = 1,inplace=False)
cdc_bts_merged_df = cdc_bts_merged_df.drop(['Level'],axis = 1,inplace=False)
cdc_bts_merged_df.to_csv('output_data/after_covid.csv', index=False)
cdc_bts_merged_df.head(3)

,state,tot_cases,new_case,tot_death,new_death,Date,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
0,CA,3570660,2234,58090,154,2021-04-01,11474065.0,28038158.0,107561279.0,29778606.0,26282896.0,12531005.0,15269950.0,14726338.0,5657090.0,2044897.0,918115.0,239073.0,113309.0
1,CA,3685032,644,62011,5,2021-05-31,10587959.0,28924264.0,121673931.0,38312879.0,30569304.0,14023600.0,15608127.0,14034739.0,5227230.0,2354560.0,1178859.0,254375.0,110258.0
2,NE,0,0,0,0,2020-02-06,354678.0,1574590.0,7285139.0,1853508.0,1951163.0,911062.0,1085616.0,899487.0,324464.0,160810.0,74008.0,16664.0,8357.0


In [42]:
#CONSOLIDATE PRE COVID DATA

bts_trip_before_covid_data = bts_trip_clean_data.loc[(bts_trip_clean_data['Date']<'2020/01/22')]
bts_trip_before_covid_data = bts_trip_before_covid_data.drop(['Level'],axis = 1,inplace=False)
bts_trip_before_covid_data = bts_trip_before_covid_data.rename(columns={"State Postal Code":"state"})  
bts_trip_before_covid_data.to_csv('output_data/before_covid.csv', index=False)

In [43]:
#CONSOLIDATE NATIONAL DATA

bts_trip_national_data = bts_trip_raw_data.loc[(bts_trip_raw_data['Level']=='National')]
bts_trip_national_data = bts_trip_national_data.drop(['Level','State FIPS','State Postal Code','County FIPS','County Name','Row ID','Week','Month'],axis='columns')
bts_trip_national_data.to_csv('output_data/national_covid.csv', index=False)
bts_trip_national_data.head(3)

,Date,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
0,2019/01/01,77433867.0,248733553.0,8.977844e+08,241667151.0,234284795.0,108078903.0,129670778.0,116904343.0,40432062.0,15686639.0,7525563.0,1806022.0,1728112.0
1,2019/01/02,61305201.0,264862219.0,1.139452e+09,291276735.0,285887315.0,138039296.0,171637514.0,167412698.0,56148976.0,17739183.0,7817044.0,1962301.0,1531219.0
2,2019/01/03,63050480.0,263116940.0,1.162753e+09,296375014.0,290074425.0,140771581.0,175775410.0,172027487.0,57632422.0,18366626.0,8124548.0,2038099.0,1567072.0


In [44]:
#COMBINE BOTH DATASETS
before_covid_df = pd.read_csv("output_data/before_covid.csv", encoding="UTF-8")
before_covid_df['tot_cases']=0
before_covid_df['tot_death']=0

after_covid_df = pd.read_csv("output_data/after_covid.csv", encoding="UTF-8")

consolidated_covid_df=after_covid_df.append(before_covid_df)

consolidated_covid_df['total_population']=consolidated_covid_df['Population Staying at Home'] + consolidated_covid_df['Population Not Staying at Home']             
consolidated_covid_df['tot_cases%']=consolidated_covid_df['tot_cases']/consolidated_covid_df['total_population'] * 100
consolidated_covid_df['tot_death%']=consolidated_covid_df['tot_death']/consolidated_covid_df['total_population'] * 100
consolidated_covid_df['Population Staying at Home%']=consolidated_covid_df['Population Staying at Home']/consolidated_covid_df['total_population'] * 100
consolidated_covid_df['Population Not Staying at Home%']=consolidated_covid_df['Population Not Staying at Home']/consolidated_covid_df['total_population'] * 100
consolidated_covid_df.sort_values(by=['Date'], inplace=True, ascending=True)
consolidated_covid_df.to_csv('output_data/consolidated_new.csv', index=False)
consolidated_covid_df.head(3)

,state,tot_cases,new_case,tot_death,new_death,Date,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,...,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500,total_population,tot_cases%,tot_death%,Population Staying at Home%,Population Not Staying at Home%
48,MO,0,NaN,0,NaN,2019-01-01,1530765.0,4576960.0,16618431.0,4033851.0,...,809030.0,290444.0,154169.0,39949.0,29170.0,6107725.0,0.0,0.0,25.062769,74.937231
27,LA,0,NaN,0,NaN,2019-01-01,1042471.0,3603260.0,13635299.0,3737562.0,...,530891.0,249210.0,97879.0,22007.0,14093.0,4645731.0,0.0,0.0,22.439332,77.560668
28,AZ,0,NaN,0,NaN,2019-01-01,1721026.0,5428700.0,18705619.0,5351386.0,...,865350.0,278933.0,190394.0,39342.0,43397.0,7149726.0,0.0,0.0,24.071216,75.928784
